In [40]:
# from langchain_
import os
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2", "true")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [42]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain.agents import initialize_agent, AgentType,load_tools

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))


In [43]:
from langchain_groq import ChatGroq

llm_groq = ChatGroq(model="Gemma2-9b-It", groq_api_key=os.getenv("GROQ_API_KEY"))

In [44]:
tools=load_tools(
    ["wikipedia"],
    llm=llm,
    # google_api_key=os.getenv("GOOGLE_API_KEY"),
)

# gfin_tool=load_tools(
#     ["google-finance"],
#     llm=llm,
#     google_api_key=os.getenv("GOOGLE_API_KEY"),
# )


agent=initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=3,
    return_intermediate_steps=True
)

agent_groq=initialize_agent(
    tools=tools,
    llm=llm_groq,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=3,
    return_intermediate_steps=True
)



In [45]:
agent.invoke("What is the capital of France?") # Google Gemini



> Entering new AgentExecutor chain...
I should use Wikipedia to find the capital of France.
Action: wikipedia
Action Input: capital of France
Observation: Page: List of capitals of France
Summary: This is a chronological list of capitals of France. The capital of France has been Paris since its liberation in 1944.



Page: Capital punishment in France
Summary: Capital punishment in France (French: peine de mort en France) is banned by Article 66-1 of the Constitution of the French Republic, voted as a constitutional amendment by the Congress of the French Parliament on 19 February 2007 and simply stating "No one can be sentenced to the death penalty" (French: Nul ne peut être condamné à la peine de mort). The death penalty was already declared illegal on 9 October 1981 when President François Mitterrand signed a law prohibiting the judicial system from using it and commuting the sentences of the seven people on death row to life imprisonment. The last execution took place by guilloti

/home/dev/Desktop/Learning/AI/graph_yt/env/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/dev/Desktop/Learning/AI/graph_yt/env/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Paris
Summary: Paris (French pronunciation: [paʁi] ) is the capital and largest city of France. With an estimated population of 2,048,472 residents in January 2025 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-most populous city in the European Union and the 30th most densely populated city in the world in 2022. Since the 17th century, Paris has been one of the world's major centres of finance, diplomacy, commerce, culture, fashion, and gastronomy. Because of its leading role in the arts and sciences and its early adaptation of extensive street lighting, it became known as the City of Light in the 19th century.
The City of Paris is the centre of the Île-de-France region, or Paris Region, with an official estimated population of 12,271,794 inhabitants in January 2023, or about 19% of the population of France. The Paris Region had a nominal GDP of €765 billion (US$1.064 trillion when adjusted for PPP) in 2021, the highest in the European Union. Accor

{'input': 'What is the capital of France?',
 'output': 'The capital of France is Paris.',
 'intermediate_steps': [(AgentAction(tool='wikipedia', tool_input='capital of France', log='I should use Wikipedia to find the capital of France.\nAction: wikipedia\nAction Input: capital of France'),
   'Page: List of capitals of France\nSummary: This is a chronological list of capitals of France. The capital of France has been Paris since its liberation in 1944.\n\n\n\nPage: Capital punishment in France\nSummary: Capital punishment in France (French: peine de mort en France) is banned by Article 66-1 of the Constitution of the French Republic, voted as a constitutional amendment by the Congress of the French Parliament on 19 February 2007 and simply stating "No one can be sentenced to the death penalty" (French: Nul ne peut être condamné à la peine de mort). The death penalty was already declared illegal on 9 October 1981 when President François Mitterrand signed a law prohibiting the judicial s

In [46]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough ,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [47]:
loader =DirectoryLoader('../data',glob="./*.txt",loader_cls=TextLoader)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=20,length_function=len)
new_docs= text_splitter.split_documents(documents=docs)

doc_strings = [doc.page_content for doc in new_docs]
# doc_strings

In [48]:
db= Chroma.from_documents(documents=new_docs, embedding=embeddings, persist_directory="../data/chroma_db")
retriver =db.as_retriever(search_kwargs={"k": 4})  
# temp_res=db.similarity_search("TXT ")
# temp_res[0].page_content

In [49]:
template="""Answer the question based on the context below try to find the thing which is mentioned in the context and answer the question as best as you can.
Context:{context}
Question: {question}
"""

prompt= PromptTemplate.from_template(template=template)

In [50]:
# retrival_chain = (
#     RunnableLambda(lambda: {"context": retriver, "question": RunnablePassthrough()})
#     | prompt
#     | llm
#     | StrOutputParser()
# )


In [51]:
def retrieval_func(input_dict):
    return {"context": retriver, "question": input_dict["question"]}

retrival_chain = (
    RunnableLambda(retrieval_func)  # Use a function instead of a lambda
    | prompt
    | llm
    | StrOutputParser()
)

question = "Why use our sample TXT files?"
result = retrival_chain.invoke({"question": question})
print(result)


The provided context doesn't explain why sample TXT files would be used. The context describes a Chroma vectorstore, embeddings using GoogleGenerativeAIEmbeddings, and search parameters. It doesn't mention anything about TXT files.


In [52]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

## WIKI TOOL

In [53]:
wiki_api=WikipediaAPIWrapper()
wiki_tool=WikipediaQueryRun(api_wrapper=wiki_api)
print(wiki_tool.run({"query":"What is the capital of France?"}))

Page: Closed-ended question
Summary: A closed-ended question is any question for which a researcher provides research participants with options from which to choose a response. Closed-ended questions are sometimes phrased as a statement that requires a response.
A closed-ended question contrasts with an open-ended question, which cannot easily be answered with specific information.

Page: What Is a Nation?
Summary: "What Is a Nation?" (French: Qu'est-ce qu'une nation ?) is an 1882 lecture by French historian Ernest Renan (1823–1892) at the Sorbonne, known for the statements that a nation is "a daily plebiscite", and that nations are based as much on what people jointly forget as on what they remember.  It is frequently quoted or anthologized in works of history or political science pertaining to nationalism and national identity. It exemplifies a contractualist understanding of the nation.

Page: France
Summary: France, officially the French Republic, is a country located primarily in 

# Tavily Search

In [55]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_search=TavilySearchResults()

tavily_search.invoke({"query":"What is the capital of France?"})

[{'title': 'List of capitals of France - Wikipedia',
  'url': 'https://en.wikipedia.org/wiki/List_of_capitals_of_France',
  'content': 'Find sources:\xa0"List of capitals of France"\xa0–\xa0news\xa0· newspapers\xa0· books\xa0· scholar\xa0· JSTOR (July 2012) (Learn how and when to remove this message)\nThis is a chronological list of capitals of France. The capital of France has been Paris since its liberation in 1944.[1]\nChronology[edit]\nTournai (before 486), current-day Belgium\nSoissons (486–936)\nLaon (936–987)\nParis (987–1419), the residence of the Kings of France, although they were consecrated at Reims. [...] Bordeaux (September 1914), the government was relocated from Paris to Bordeaux very briefly during World War I, when it was feared that Paris would soon fall into German hands. These fears were alleviated after the German Army was pushed back at the First Battle of the Marne.\nTours (10–13 June 1940), the city served as the temporary capital of France during World War II 

# Prompt Template

In [56]:
from langchain import hub

tools=[tavily_search, wiki_tool]

prompt_agent=hub.pull("hwchase17/openai-functions-agent")

In [ ]:
prompt_agent.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [57]:
from langchain.agents import create_tool_calling_agent,AgentExecutor

agent_new= create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_agent,
    # verbose=True
)

In [58]:
agent_executor=AgentExecutor(
    agent=agent_new,
    tools=tools,
    verbose=True,
    # max_iterations=3,
    # return_intermediate_steps=True
)

In [59]:
agent_executor.invoke({"input":"What is the current price of telsa? also giev me overview of the company"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'tesla stock price'}`


[{'title': 'TSLA Stock Price — Tesla Chart - TradingView', 'url': 'https://www.tradingview.com/symbols/NASDAQ-TSLA/', 'content': "What is Tesla stock price today?\nThe current price of TSLA is 263.55\xa0USD — it has decreased by −3.51% in the past 24 hours. Watch Tesla stock price performance more closely on the chart.\nWhat is Tesla stock ticker?\nDepending on the exchange, the stock ticker may vary. For instance, on NASDAQ exchange Tesla stocks are traded under the ticker TSLA.\nIs Tesla stock price growing? [...] What are Tesla stock highest and lowest prices ever?\nTSLA reached its all-time high on Dec 18, 2024 with the price of 488.54\xa0USD, and its all-time low was 1.00\xa0USD and was reached on Jul 7, 2010. View more price dynamics on TSLA chart.\nSee other stocks reaching their highest and lowest prices.\nHow volatile is TSLA stock?\nTSLA stock is 5.96% volat

/home/dev/Desktop/Learning/AI/graph_yt/env/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/dev/Desktop/Learning/AI/graph_yt/env/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Tesla, Inc.
Summary: Tesla, Inc. ( TEZ-lə or   TESS-lə) is an American multinational automotive and clean energy company. Headquartered in Austin, Texas, it designs, manufactures and sells battery electric vehicles (BEVs), stationary battery energy storage devices from home to grid-scale, solar panels and solar shingles, and related products and services.
Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors. Its name is a tribute to inventor and electrical engineer Nikola Tesla. In February 2004, Elon Musk led Tesla's first funding round and became the company's chairman; in 2008, he was named chief executive officer. In 2008, the company began production of its first car model, the Roadster sports car, followed by the Model S sedan in 2012, the Model X SUV in 2015, the Model 3 sedan in 2017, the Model Y crossover in 2020, the Tesla Semi truck in 2022 and the Cybertruck pickup truck in 2023.
Tesla is one of the world's most valuable companies

{'input': 'What is the current price of telsa? also giev me overview of the company',
 'output': "Okay, here's an overview of Tesla and its stock price:\n\n**Current Stock Price:**\n\nAccording to TradingView, the current price of TSLA is $263.55 USD, a decrease of 3.51% in the past 24 hours.\n\n**Tesla, Inc. Overview:**\n\n*   **Company Type:** American multinational automotive and clean energy company.\n*   **Headquarters:** Austin, Texas.\n*   **Business:** Designs, manufactures, and sells battery electric vehicles (BEVs), stationary battery energy storage, solar panels/shingles, and related services.\n*   **Founding:** Founded in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors.\n*   **Name:** Named after inventor and electrical engineer Nikola Tesla.\n*   **Key People:** Elon Musk is the chairman and CEO.\n*   **Vehicles:** Known for models like the Roadster, Model S, Model X, Model 3, Model Y, Semi, and Cybertruck.\n*   **Market Value:** One of the world's most va

In [ ]:
from langchain_community.document_loaders import  WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


### Webloader 

In [60]:
loader=WebBaseLoader("https://www.screener.in/company/HINDUNILVR/")
stock_data=loader.load()
stock_data[0].page_content

"\n\n\n\n\n\nHindustan Unilever Ltd share price | About Hind. Unilever | Key Insights - Screener\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        Home\n      \n\n\n\n\n\n      Screens\n    \n\n\n\n\n\n\n\n\n\n\n      Tools\n    \n\n\n\n\n\n        Login\n      \n\n\n\n\n\n\n\n\n\n\n\n\n          \n            Home\n          \n        \nScreens\n\n\n            Tools\n            \n\n\n\n\n\n\n\n\nCreate a stock screen\nRun queries on 10 years of financial data\n\n\n\n\n\n\nPremium features\n\n\n\n\n\n\n\n\n\nCommodity Prices\nSee prices and trends of over 10,000 commodities\n\n\n\n\n\n\n\n\n\nSearch shareholders\nSee companies where a person holds over 1% of the shares\n\n\n\n\n\n\n\n\n\nLatest Announcements\nBrowse, filter and set alerts for announcements.\n\n\n\n\nUpgrade to premium\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Login\n          \nGet free account\n\n\n\n\n\n\n\n\n\nHindustan Unilever Ltd\n\n\n\nNotebook\n\n\n\n\n\n\nHind. Unilever\nSummary

In [ ]:
final_docs=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(stock_data)

In [ ]:
vectordb=Chroma.from_documents(documents=final_docs, embedding=embeddings)


In [61]:
retriver_stock=vectordb.as_retriever()
retriver_stock.invoke("What is the current price of HINDUNILVR")

[Document(metadata={'language': 'en', 'source': 'https://www.screener.in/company/HINDUNILVR/consolidated/', 'title': 'Hindustan Unilever Ltd share price | About Hind. Unilever | Key Insights - Screener'}, page_content='Notebook\n\n\n\n\n\n\nHind. Unilever\nSummary\n\nChart\nAnalysis\nPeers\nQuarters\nProfit & Loss\nBalance Sheet\nCash Flow\nRatios\nInvestors\nDocuments\n\n\n\n\nNotebook\n\n\n\n\n\n\n\n\n\n\n\nHindustan Unilever Ltd\n\n\n\n\n\n₹ 2,245\n\n\n                0.00%\n              \n\n\n            \n              04 Apr\n              \n                - close price\n              \n            \n          \n\n\n\n\n\n\n\nExport to Excel\n\n\n\n Follow\n            \n\n\n\n\n\n\nhul.co.in\n\n\n\n\n            BSE:\n            500696\n          \n\n\n\n\n            NSE:\n            HINDUNILVR\n          \n\n\n\n\n\nAbout\nHindustan Unilever is in the FMCG business comprising primarily of Home Care, Beauty & Personal Care and Foods & Refreshment segments. The Company has m

In [62]:
from langchain.tools.retriever import create_retriever_tool
stock_retriever_tool = create_retriever_tool(
    retriever=retriver_stock,
    name="stock_retriever",
    description="Retrieves information about stocks financial data, fundamentals, and other related information.",
    # verbose=True,
)

In [ ]:
stock_tools=[tavily_search, wiki_tool, stock_retriever_tool]

In [64]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

stock_agent= create_tool_calling_agent(
    llm=llm,
    tools=stock_tools,
    prompt=prompt_agent,
    # verbose=True
)

stock_agent_executor=AgentExecutor(
    agent=stock_agent,
    tools=stock_tools,
    verbose=True,
    # max_iterations=3,
    return_intermediate_steps=True
)
stock_agent_executor.invoke({"input":"What is the recent profit of Adani Enterprise? Search this from net and compare with my retrieval tool data and tell me the difference."})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Adani Enterprise recent profit'}`


[{'title': 'Adani Enterprises Ltd Results 2024 - INDmoney', 'url': 'https://www.indmoney.com/stocks/adani-enterprises-ltd-share-price/results', 'content': "Net Profits: Adani Enterprises Ltd's net profit fell -96.94% since last year same period to ₹57.83Cr in the Q3 2024-2025. On a quarterly growth basis,", 'score': 0.8832463}, {'title': 'Adani Ent. Share Price - Stocks - The Economic Times', 'url': 'https://economictimes.indiatimes.com/adani-enterprises-ltd/stocks/companyid-9074.cms', 'content': 'On Consoldiated basis, Adani Ent. reported a total income and profit of Rs 23500.54 Cr and Rs 57.83 respectively for quarter ending 2024-12-31. Total Income and', 'score': 0.86713725}, {'title': 'Adani Enterprises Q3 Results: Profit falls 97% YoY to Rs 58 crore', 'url': 'https://m.economictimes.com/markets/stocks/earnings/adani-enterprises-q3-results-profit-near

{'input': 'What is the recent profit of Adani Enterprise? Search this from net and compare with my retrieval tool data and tell me the difference.',
 'output': 'Okay, I have the information from both the search engine and the stock retriever tool.\n\nBased on the search results, Adani Enterprises\' net profit for the latest quarter (Q3 2024-2025) is approximately ₹57.83 crore. However, this represents a significant decrease of around 97% compared to the same period last year.\n\nThe stock retriever provides a lot of financial data, but it doesn\'t give me a single, recent profit figure to directly compare with the ₹57.83 crore. It does show a table of "Net Profit" in "Rs. Crores" with the most recent value being 2,989. This seems to be an annual figure.\n\nTherefore, I can\'t give you a precise difference between the search result and the retriever data. The search result gives the profit for Q3 2024-2025, while the retriever provides annual net profit figures. To compare accurately, I

### Google Serper API

In [ ]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.tools  import Tool 
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

import os
from dotenv import load_dotenv
load_dotenv()
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")


66aea2a3e09ae006735af6d40ee743b05b7c503c2389cc5f8ff1b1f57d125855


In [73]:
google_search=GoogleSerperAPIWrapper(serper_api_key=os.getenv("SERPAPI_API_KEY"))
google_search=Tool(
    name="google_search",
    func=google_search.run,
    description="Use this tool to search the web for information."
)

final_search_tool=[tavily_search, wiki_tool, stock_retriever_tool, google_search]

In [76]:
template=""" Answer the best as you can us the tools the tool to search the web for information.
{tools}
use the following format:
Question:the input question you must answer
Thought: you should always think about what to do next
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
...(this Thought/Action/Action Input/Observation can repeat N times)
Final Observation: the final observation of the last action
Thought:Now i now know the final answer
Final Answer: the final answer to the question
Begin!
Question: {input}
Thought: {agent_scratchpad}"""

In [77]:
prompt=PromptTemplate.from_template(template=template)
google_agent= create_react_agent(
    llm=llm,
    tools=final_search_tool,
    prompt=prompt,
    # verbose=True
)
google_agent_executor=AgentExecutor(
    agent=google_agent,
    tools=final_search_tool,
    verbose=True,
    # max_iterations=3,
    return_intermediate_steps=True
)


In [78]:
google_agent_executor.invoke({"input":"What is the recent profit of Adani Enterprise? Search this from net and compare with my retrieval tool data and tell me the difference."})



> Entering new AgentExecutor chain...
Okay, I need to find the recent profit of Adani Enterprises. I will first use the web search tool to get the latest news on Adani Enterprises' profits. Then I will use the stock retriever tool to get the profit data, and compare both. Finally, I will calculate the difference.

Action: tavily_search_results_json
Action Input: "Adani Enterprises recent profit"[{'title': 'Adani Enterprises Ltd. Share Price Today, Market ... - Finology Ticker', 'url': 'https://ticker.finology.in/company/ADANIENT', 'content': 'Earnings Per Share: - It tells us how much profit is allocated to\xa0to each outstanding share of a common stock.\xa0The latest EPS of\xa0Adani Enterprises is Rs 22.1201 . The higher the EPS, the better it is for investors.\n\nOne can find all the Financial Ratios of\xa0Adani Enterprises\xa0in Ticker for free. Also, one can get the intrinsic value of Adani Enterprises by using Valuation Calculators, which are available with a Finology ONE subscr

{'input': 'What is the recent profit of Adani Enterprise? Search this from net and compare with my retrieval tool data and tell me the difference.',
 'output': "Adani Enterprises' recent profit for Q3FY25 is reported as Rs 58 crore (Rs 0.058 billion) according to news articles. The stock retriever tool shows a TTM net profit of Rs 10,757 crore. The difference between the TTM net profit and the recent Q3FY25 profit is Rs 10,699 crore.",
 'intermediate_steps': [(AgentAction(tool='tavily_search_results_json', tool_input='Adani Enterprises recent profit', log='Okay, I need to find the recent profit of Adani Enterprises. I will first use the web search tool to get the latest news on Adani Enterprises\' profits. Then I will use the stock retriever tool to get the profit data, and compare both. Finally, I will calculate the difference.\n\nAction: tavily_search_results_json\nAction Input: "Adani Enterprises recent profit"'),
   [{'title': 'Adani Enterprises Ltd. Share Price Today, Market ... -